## Кейс Avito

Обучите классификатор, предсказывающий категорию объявления на Авито по его заголовку, описанию и цене. Метрика для оценки качества -- accuracy. Необходимо предоставить прокомментированный код (желательно на Python 2.x или 3.x, можно в Jupiter Notebook) для всех этапов решения задачи и результат скоринга файла test.csv с помощью предложенного классификатора (csv-файл с двумя столбцами: item_id, category_id).

Категории имеют иерархическую структуру, описанную в файле сategory.csv. Посчитайте также accuracy вашей модели на каждом уровне иерархии.

Для решения задачи можно использовать любые внешние модели, но не внешние данные. В случае сомнений по поводу возможности использования чего-либо напишите нам.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import seaborn as sns
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import nltk

In [2]:
# посмотрим на данные

In [3]:
raw_df = pd.read_csv('train.csv')
raw_df.head(5)

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489517 entries, 0 to 489516
Data columns (total 5 columns):
item_id        489517 non-null int64
title          489517 non-null object
description    489517 non-null object
price          489517 non-null float64
category_id    489517 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 18.7+ MB


In [5]:
raw_df.item_id.value_counts().head()

2047      1
216259    1
60599     1
38072     1
40121     1
Name: item_id, dtype: int64

In [6]:
# видимо просто идентификатор записи - выбросим для обучения

In [7]:
raw_df.category_id.value_counts().head()

49    9998
18    9932
29    9904
2     9887
37    9884
Name: category_id, dtype: int64

In [8]:
raw_df.category_id.value_counts().tail()

30    8223
25    8106
9     8033
1     8022
40    8009
Name: category_id, dtype: int64

In [9]:
# классы в основном сбалансированные - нет необходимости использовать техники по балансировке

In [10]:
len(raw_df.title.value_counts())

365152

In [11]:
raw_df.describe()

,item_id,price,category_id
count,489517.000000,4.895170e+05,489517.000000
mean,244758.000000,8.795970e+03,26.765361
std,141311.530199,7.158194e+04,15.531774
min,0.000000,1.300000e+01,0.000000
25%,122379.000000,7.500000e+02,13.000000
50%,244758.000000,2.400000e+03,27.000000
75%,367137.000000,7.000000e+03,40.000000
max,489516.000000,1.000000e+07,53.000000


In [12]:
category = pd.read_csv('category.csv')
category.head(5)

,category_id,name
0,0,Бытовая электроника|Телефоны|iPhone
1,1,Бытовая электроника|Ноутбуки
2,2,Бытовая электроника|Телефоны|Samsung
3,3,Бытовая электроника|Планшеты и электронные кни...
4,4,"Бытовая электроника|Игры, приставки и программ..."


In [13]:
# реализуем стандартную предобработку текстовых данных - оставим только слова, 
# уберем стоп-слова, лемматизируем, уберем стоп-леммы 

In [195]:
from nltk.corpus import stopwords

In [196]:
mystopwords = stopwords.words('russian') + ['-', 'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'г']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

In [197]:
from pymystem3 import Mystem
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [198]:
import re
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [199]:
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш', 'мочь']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return " "

In [19]:
raw_df['title_lemma']= raw_df.title.apply(words_only)
raw_df['title_lemma']= raw_df['title_lemma'].apply(remove_stopwords)
raw_df['title_lemma']= raw_df['title_lemma'].apply(lemmatize)
raw_df['title_lemma']= raw_df['title_lemma'].apply(remove_stoplemmas)

In [20]:
raw_df['desc_lemma']= raw_df.description.apply(words_only)
raw_df['desc_lemma']= raw_df['desc_lemma'].apply(remove_stopwords)
raw_df['desc_lemma']= raw_df['desc_lemma'].apply(lemmatize)
raw_df['desc_lemma']= raw_df['desc_lemma'].apply(remove_stoplemmas)

In [193]:
raw_test = pd.read_csv('test.csv')

In [194]:
# аналогичную предобработку делаем для теста

In [200]:
raw_test['title_lemma']= raw_test.title.apply(words_only)
raw_test['title_lemma']= raw_test['title_lemma'].apply(remove_stopwords)
raw_test['title_lemma']= raw_test['title_lemma'].apply(lemmatize)
raw_test['title_lemma']= raw_test['title_lemma'].apply(remove_stoplemmas)

In [201]:
raw_test['desc_lemma']= raw_test.description.apply(words_only)
raw_test['desc_lemma']= raw_test['desc_lemma'].apply(remove_stopwords)
raw_test['desc_lemma']= raw_test['desc_lemma'].apply(lemmatize)
raw_test['desc_lemma']= raw_test['desc_lemma'].apply(remove_stoplemmas)

In [187]:
raw_df.head(3)

,item_id,title,description,price,category_id,title_lemma,desc_lemma
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,картина,гобелен размер х см
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,стул прессованный кожа,продавать недорого стул светлый прессованный к...
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,домашний мини баня,мини баня мб мини сауна предназначать принятие...


In [202]:
raw_test.head(3)

,item_id,title,description,price,title_lemma,desc_lemma
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0,стоик журнальный сталь,продавать журнальный столик изготовлять стол п...
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0,iPhone Gb,телефон хороший состояние комплект гарантия са...
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0,утеплитель,теплопельный лидер тепло толщина утеплитель мм...


In [217]:
raw_test.shape

(243166, 6)

In [27]:
df = raw_df[['price', 'title_lemma', 'desc_lemma', 'category_id']]

In [28]:
df_test = raw_test[['price', 'title_lemma', 'desc_lemma']]

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression

In [30]:
X = df[['price', 'title_lemma', 'desc_lemma']]

In [31]:
def concate(row):
    d = str(row['price']) + ' ' + row['title_lemma']+' ' + row['desc_lemma']
    return d

In [32]:
X['text'] = X.apply(concate, axis=1)

In [ ]:
X = X.text
y = df['category_id']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# запустим пайплайн обучения данных - предварительная обработка 
#векторайзером со словами размерностью 3-6 символов, потом применим tf-idf разложение

In [39]:
clf = Pipeline([ 
    ('vect', CountVectorizer(analyzer = 'char', ngram_range={3,6})), 
    ('tfidf', TfidfTransformer()),   
    ('clf', RandomForestClassifier(n_jobs = 3))
])

In [40]:
clf.fit(X_train, y_train)

/home/pavel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range={3, 6}, preprocessor=None, stop_words=None,
        strip...n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [41]:
y_pred = clf.predict(X_test)

In [42]:
from sklearn.metrics import *

In [43]:
print("Precision: {0:6.2f}".format(precision_score(y_test, y_pred, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, y_pred, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, y_pred, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
labels = clf.classes_

Precision:   0.79
Recall:   0.80
F1-measure:   0.79
Accuracy:   0.80
              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1765
           1       0.81      0.90      0.85      1625
           2       0.76      0.88      0.81      2017
           3       0.85      0.85      0.85      1729
           4       0.82      0.89      0.86      1760
           5       0.89      0.91      0.90      1666
           6       0.64      0.70      0.67      1712
           7       0.88      0.89      0.88      1700
           8       0.79      0.88      0.83      1778
           9       0.79      0.83      0.81      1594
          10       0.88      0.86      0.87      1770
          11       0.91      0.90      0.91      1937
          12       0.80      0.88      0.84      1834
          13       0.92      0.93      0.93      1772
          14       0.89      0.90      0.89      1684
          15       0.58      0.73      0.65      1800
          16

In [60]:
y_test = pd.DataFrame(y_test)
y_pred = pd.DataFrame(y_pred)

In [61]:
y_test = y_test.merge(category, on = 'category_id', how = 'left')

In [86]:
y_test.head()

,category_id,name
0,10,Бытовая электроника|Телефоны|Nokia
1,49,Хобби и отдых|Охота и рыбалка
2,35,"Личные вещи|Одежда, обувь, аксессуары|Женская ..."
3,31,"Личные вещи|Одежда, обувь, аксессуары|Аксессуары"
4,5,Бытовая электроника|Аудио и видео|Телевизоры и...


In [68]:
y_pred = y_pred.merge(category, on = 'category_id', how = 'left')

In [111]:
def ifnull(var, val):
      if var is None:
        return val
      return var

In [171]:
def cat_split(df):
    d = df['name'].split('|') 
    if len(d) < 4:
        d.append('')
    if len(d) < 3:
        d.append('')
    return d[0]

In [172]:
def cat_split1(df):
    d = df['name'].split('|') 
    if len(d) < 4:
        d.append('')
    if len(d) < 3:
        d.append('')
    return d[1]

In [173]:
def cat_split2(df):
    d = df['name'].split('|') 
    if len(d) < 4:
        d.append('')
    if len(d) < 3:
        d.append('')
    return d[2]

In [174]:
def cat_split3(df):
    d = df['name'].split('|') 
    if len(d) < 4:
        d.append('')
    if len(d) < 3:
        d.append('')
    return d[3]

In [176]:
y_test['cat1']  = y_test.apply(cat_split, axis = 1)
y_pred['cat1'] = y_pred.apply(cat_split, axis = 1)

In [177]:
y_test['cat2'] = y_test.apply(cat_split1, axis = 1)
y_pred['cat2'] = y_pred.apply(cat_split1, axis = 1)

In [178]:
y_test['cat3'] = y_test.apply(cat_split2, axis = 1)
y_pred['cat3'] = y_pred.apply(cat_split2, axis = 1)

In [180]:
y_test['cat4'] = y_test.apply(cat_split2, axis = 1)
y_pred['cat4'] = y_pred.apply(cat_split2, axis = 1)

In [181]:
print("Accuracy cat1: {0:6.2f}".format(accuracy_score(y_test['cat1'], y_pred['cat1'])))
print("Accuracy cat2: {0:6.2f}".format(accuracy_score(y_test['cat2'], y_pred['cat2'])))
print("Accuracy cat3: {0:6.2f}".format(accuracy_score(y_test['cat3'], y_pred['cat3'])))
print("Accuracy cat4: {0:6.2f}".format(accuracy_score(y_test['cat4'], y_pred['cat4']))) 

Accuracy cat1:   0.91
Accuracy cat2:   0.87
Accuracy cat3:   0.81
Accuracy cat4:   0.81


In [ ]:
# Для категорий верхнего уровня Accuracy - выше. Для первой категории - 91%

In [203]:
raw_test2 = raw_test[['item_id', 'price', 'title_lemma', 'desc_lemma']]

In [206]:
raw_test2['text'] = raw_test2.apply(concate, axis=1)

/home/pavel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [208]:
X_send = raw_test2[['item_id', 'text']]

In [47]:
y_send = clf.predict(X_send['text'])

In [209]:
X_send.head()

,item_id,text
0,489517,10000.0 стоик журнальный сталь продавать журна...
1,489518,12500.0 iPhone Gb телефон хороший состояние ко...
2,489519,250.0 утеплитель теплопельный лидер тепло толщ...
3,489520,1700.0 пальто демисезонный продавать пальто же...
4,489521,1000.0 Samsung syncmaster T N условно рабочий ...


In [183]:
y_send = pd.DataFrame(y_send)

In [211]:
send = pd.concat([X_send, y_send], axis =1)

In [213]:
send = send[['item_id', 0]]

In [214]:
send.columns = ['item_id', 'category_id']

In [216]:
send.shape

(243166, 2)

In [218]:
send.to_csv('avito_predict.csv')

In [ ]:
# xgboost к сожалению не запустился на моих мощностях